### 아이디어
- 트리에서 `u` -> `v`의 거리 및 경로 = `u` -> LCA(u, v) -> `v`
  - 경로의 비용은 `u -> LCA(u, v)`의 거리의 합 + `LCA(u, v) -> v`의 거리의 합
  - `k`번째 정점은 `k`가 `u와 LCA(u, v) 까지의 거리`보다
    - 짧거나 같으면 `u`의 `k`번째 조상을 출력
    - 길면 `v`의 `(v와 LCA(u, v)까지의 거리) - (k - (u와 LCA(u, v)까지의 거리))`번째 조상을 출력

In [ ]:
import sys, io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

N = int(input())
G = [[] for _ in range(N+1)]
for _ in range(N-1) :
  u, v, w = map(int, input().split())
  G[u].append((v, w))
  G[v].append((u, w))

def dfs(G, P, L, W) :
  vis = [False] * (N+1)
  vis[1] = True
  Q = [1]
  l = 1
  while Q :
    nextQ = []
    while Q:
      u = Q.pop()
      for v, w in G[u] :
        if not vis[v] :
          vis[v] = True
          P[0][v] = u
          W[0][v] = w
          L[v] = l
          nextQ.append(v)
    Q = nextQ
    l += 1

MAX = 17
L = [0] * (N+1)
P = [[0] * (N+1) for _ in range(MAX)]
W = [[0] * (N+1) for _ in range(MAX)] #[i][j] : j의 2^i번째 부모까지 거리의 합
dfs(G, P, L, W)

for i, l in enumerate(P[1:], 1) :
  for j in range(N+1) :
    tmp = P[i-1][j]
    l[j] = P[i-1][tmp]
    W[i][j] = W[i-1][j] + W[i-1][tmp]

for _ in range(int(input())) :
  Q = tuple(map(int, input().split()))

  if Q[0] == 1 :
    a, b = Q[1:]
    
    if L[a] < L[b] :
      a, b = b, a

    w = 0
    for i in range(MAX-1, -1, -1) :
      if L[a] - L[b] >= (1 << i) :
        w += W[i][a]
        a = P[i][a]
    
    if a != b:
      for i in range(MAX-1, -1, -1) :
        if P[i][a] != -1 and P[i][a] != P[i][b] :
          w += W[i][a] + W[i][b]
          a = P[i][a]
          b = P[i][b]
      w += W[0][a] + W[0][b]
    sys.stdout.write(str(w) + '\n')

  if Q[0] == 2 :
    a, b, k = Q[1:]

    swapped = False
    if L[a] < L[b] :
      swapped = True
      a, b = b, a

    a_l = 0 #a에서 LCA(a, b)까지의 거리
    for i in range(MAX-1, -1, -1) :
      if L[a] - L[b] >= (1 << i) :
        a_l += (1 << i)
        a = P[i][a]
    
    b_l = 0
    if a != b:
      for i in range(MAX-1, -1, -1) :
        if P[i][a] != -1 and P[i][a] != P[i][b] :
          a_l += (1 << i)
          b_l += (1 << i)
          a = P[i][a]
          b = P[i][b]
      a_l += 1
      b_l += 1
    a_l, b_l = (b_l, a_l) if swapped else (a_l, b_l) #원래 입력순서에 맞게 다시 바꾼다.

    k -= 1 #1번째 = 자기자신
    if k <= a_l :
      a = Q[1]
      for i in range(MAX-1, -1, -1) : #a의 k번째 조상을 출력한다.
        if k >= (1 << i) : 
          a = P[i][a]
          k -= (1 << i)
      sys.stdout.write(str(a) + '\n')
    else :
      b = Q[2]
      k = b_l + a_l - k
      for i in range(MAX-1, -1, -1) : #b의 (b_l + a_l - k)번째 조상을 출력한다.
        if k >= (1 << i) :
          b = P[i][b]
          k -= (1 << i)
      sys.stdout.write(str(b) + '\n')

- 한방치료 시원하게 성공
- 1번째 경로가 0번째 조상인 부분만 조심하면 됐을거 같다.